# Training a LSTM model without pre-training
for off-the-record prediction

In [1]:
import os
import sys
import json
import logging

os.chdir('../')

Make sure that your current working directory (cwd) is `ReproducingAugSS/AugmentedSocialScientist/`

In [116]:
#os.getcwd() 

In [117]:
from PATHS import OFF_ASS, OFF_GS, SAVED_MODELS_PATH

In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.cuda import empty_cache
from torch import masked_select


from TransferSociologist.data import Dataset
from TransferSociologist.models import BertSequence
from TransferSociologist.utils import regularize_seqlab

import pandas as pd
import numpy as np
import random
from sklearn.metrics import precision_recall_fscore_support,classification_report

from tqdm.auto import tqdm

from operator import add
from functools import reduce
from copy import deepcopy


from statistics import harmonic_mean
import statsmodels.stats.api as sms

## Preliminary functions

In [49]:
def try_eval(x):
    try:
        return eval(x)
    except:
        return x


def fill_zeros(labels, zeros, conv_dict):
    try:
        labels = eval(labels)
    except:
        pass
    for l in labels:
        start_span, stop_span, lab = l
        size = len(zeros[start_span:stop_span])
        number = conv_dict[lab]
        zeros[start_span:stop_span] = [number] * size
    return zeros


In [118]:
# Defining a LSTM model for token classification
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size, num_layers=1):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, 
                                            embedding_dim,
                                            padding_idx=1)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        #print(embeds.shape)
        lstm_out, _ = self.lstm(embeds)
        #print(lstm_out.shape)
        tag_space = self.hidden2tag(lstm_out)#.view(len(sentence), -1))
        #print(tag_space.shape)
        tag_scores = F.log_softmax(tag_space, dim=2)
        #print(tag_scores.shape)
        return tag_scores

In [51]:
def prepare_experiment(
    train_path, gs_path, drop_duplicates=False, percent_of_data=1
):
    dataset = Dataset()
    dataset.read(
        data_path=train_path, gold_standard_path=gs_path, data_type="csv"
    )
    dataset.df = dataset.df.rename({'is_control_1': 'is_control'}, axis=1)
    if drop_duplicates == True:
        if "is_control" in dataset.df.columns:
            gs = dataset.df[dataset.df.is_gold_standard == True]
            no_gs = dataset.df[dataset.df.is_gold_standard == False]
            no_gs = pd.concat(
                [
                    no_gs[no_gs.is_control == True]
                        .groupby(["text"])
                        .apply(lambda x: x.sample(1))
                        .reset_index(drop=True),
                    no_gs[no_gs.is_control != True]
                ]
            )
            dataset.df = pd.concat([no_gs, gs])
    # Now sample subset of data
    gs = dataset.df[dataset.df.is_gold_standard == True]
    no_gs = dataset.df[dataset.df.is_gold_standard == False]
    no_gs = no_gs.sample(frac=percent_of_data)
    dataset.df = pd.concat([no_gs, gs])

    dataset.task_encode(task_type="sequence_labelling", bert_model="CamemBert")
    # natural_samples = dataset.df

    dataset.encode_torch(
        task_type="sequence_labelling",
        bert_model="CamemBert",
        # test_size=0.3,
        random_seed=2018,
    )

    dataset_pred = Dataset()
    dataset_pred.read(data_path=gs_path, data_type="csv")
    dataset_pred.task_encode(
        task_type="sequence_labelling",
        bert_model="CamemBert",
        # pred_gs=True,
        pred_mode=True,
    )
    dataset_pred.encode_torch(
        task_type="sequence_labelling", bert_model="CamemBert", pred_mode=True
    )
    return dataset, dataset_pred


In [52]:
def do_batch(train, pred=False, test=None, batch_size=1, nb_labels=2, sampler='random'):
    # Decompose train and test as subvectors
    if pred==True:
        train_inputs, train_masks = train
        train_inputs = torch.tensor(train_inputs, dtype=torch.long)
        train_masks = torch.tensor(train_masks, dtype=torch.long)
        train_data = TensorDataset(train_inputs, train_masks)
    else:
        train_inputs, train_labels, train_masks = train
        train_inputs = torch.tensor(train_inputs, dtype=torch.long)
        train_masks = torch.tensor(train_masks, dtype=torch.long)
        train_labels = torch.tensor(train_labels, dtype=torch.long)
        train_data = TensorDataset(train_inputs, train_masks, train_labels)

    if sampler=='sequential':
        train_sampler = SequentialSampler(train_data)
    elif sampler=='random':
        train_sampler = RandomSampler(train_data)
    else:
        raise NotImplementedError
    train_dataloader = DataLoader(train_data, 
                                sampler=train_sampler,
                                batch_size=batch_size)
    out = [train_dataloader]
    
    if test is not None:
        validation_inputs, validation_labels, validation_masks = test
        validation_inputs = torch.tensor(validation_inputs, dtype=torch.long)
        validation_labels = torch.tensor(validation_labels, dtype=torch.long)
        validation_masks = torch.tensor(validation_masks, dtype=torch.long)
        # Create the DataLoader for our validation set.
        # len of val dataset
        validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
        validation_sampler = SequentialSampler(validation_data)
        validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)
        out.append(validation_dataloader)
    
    return out

In [53]:
def train_loop(dataloader, model, optimizer):
    size = len(dataloader.dataset)
    device = torch.device("cuda")
    model.to(device)
    for batch, (X, masks, y) in enumerate(dataloader):
        X = X.to(device)
        masks = masks.to(device)
        y = y.to(device)
        masks = masks-1
        # Compute prediction and loss
        pred = model(X).view(-1, 2)
        y = (y+masks).view(-1)
        
        loss = nn.NLLLoss(ignore_index=-1)(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    pred_flat_all = []
    y_flat_all = []
    model.to('cpu')
    with torch.no_grad():
        for batch, (X, masks, y) in enumerate(dataloader):
            masks = masks.gt(0.5)
            pred = model(X)
            pred = pred.argmax(2)
            pred_flat_all = pred_flat_all + list(masked_select(pred, masks))
            y_flat_all = y_flat_all + list(masked_select(y, masks))
    print(classification_report(y_flat_all, pred_flat_all))
    return precision_recall_fscore_support(y_flat_all, pred_flat_all)



def predict_loop(dataloader, model):  
    pred_flat_all = []
    model.to('cpu')
    with torch.no_grad():
        for batch, (X, masks) in enumerate(dataloader):
            masks = masks.gt(0.5)
            pred = model(X)
            pred = pred.argmax(2)
            ls_batch_preds = []
            for i, line in enumerate(pred):
                sent_pred = list(line.numpy()[: masks[i].sum()])
                ls_batch_preds.append(sent_pred)
            pred_flat_all = pred_flat_all + ls_batch_preds
    return pred_flat_all

In [66]:
#post treatments of predictions

def merge(preds):
    if len(preds)<=1:
        return preds
    else:
        list_labels = []
        i = 0
        while i < len(preds):
            if (i<len(preds)-1) and ((preds[i+1][0]<=preds[i][1]) or ((preds[i+1][0] - preds[i][1])<15)):# or ((preds[i+1][0] - preds[i][1])<20):
                list_labels.append([preds[i][0],max(preds[i][1],preds[i+1][1]),'off'])
                i += 2
            else:
                list_labels.append(preds[i])
                i+=1

        return list_labels
    
    
def delet_short(labels, n_chars=5):
    list_labels=[]
    for label in labels:
        if label[1]-label[0] >= n_chars:
            list_labels.append(label)
    return list_labels    

In [119]:
# Function to count the number of ture positive according to the metric "by Span"
def count_tp(pr, gs, threshold=.5):
    # Inputs: 
    #     pr: list of the spans annotated by the annotator
    #     gs : list of the spans in the gold standard
    # Output: the number of true positives
    tp = 0 
    for i, pred_labels in enumerate(pr):
        for pred_lab in pred_labels:
            #check if pred_lab is a true positive
            for gs_lab in gs.iloc[i]:
                ## if at least a proportion (larger than the threshold) of the gs span is correctly identified 
                if len(set(range(pred_lab[0],pred_lab[1])) & set(range(gs_lab[0],gs_lab[1]))) >= threshold*(gs_lab[1]-gs_lab[0]):
                    tp+=1
    return tp

## Main training function

In [88]:
def run_all(random_state=42):
    
    dict_scores = {'random_state':random_state}
    
    # setting random seed
    random.seed(random_state)
    np.random.seed(random_state)
    torch.manual_seed(random_state)
    torch.cuda.manual_seed_all(random_state)
    
    empty_cache()

    ### preparing datasets
    train_path = OFF_ASS
    gs_path = OFF_GS
 
    drop_duplicates = True
    percent_of_data = 1
    
    dataset, dataset_pred = prepare_experiment(
        train_path, gs_path, drop_duplicates, percent_of_data
    )
    
    ### training
    
    #training parameters
    batch_size=528  
    lr = 0.01
    nepochs = 7
    nlayers = 2
    emb_dim = 700
    hidden_dim = 50

    train_unique_labels = reduce(lambda x, y :list(set(x+y)), dataset.train[0])
    test_unique_labels = reduce(lambda x, y :list(set(x+y)), dataset.test[0])
    unique_labels = list(set(train_unique_labels+test_unique_labels))
    vocab_size=max(unique_labels)+1

    # defining model
    train, test = do_batch(dataset.train, test=dataset.test, batch_size=batch_size)
    pred = do_batch(dataset_pred.pred, pred=True, batch_size=batch_size, sampler = 'sequential')[0]
    model = LSTMTagger(emb_dim, hidden_dim, vocab_size, 2, num_layers=nlayers)
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    device = torch.device("cuda")
    
    # running training loops
    for t in range(nepochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train_loop(train, model, optimizer)
        perfs = test_loop(test, model)
    
    ### predicting
    truncated_labels = predict_loop(pred, model)
    dataset_pred.df["truncated_labels"] = truncated_labels
    dataset_pred = regularize_seqlab(dataset_pred, dataset.tokenizer)

    ### post treatments of predictions
    df_pred = dataset_pred.df[['id','sents','labels','pred_labels']]

    #sorting labels
    df_pred.labels = df_pred.labels.apply(try_eval).apply(lambda x: sorted(x, key=lambda y: y[0]))
    df_pred.pred_labels = df_pred.pred_labels.apply(lambda x: sorted(x, key=lambda y: y[0]))


    # merging pred labels if they are separated by too few characters
    # The LSTM model often predicts a sequence of short labels seperated by very few characters.
    # We decide to merge predicted spans if they are separated by too few characters (<15)
    total_labels = 0
    new_total_labels = df_pred.pred_labels.apply(len).sum()

    while new_total_labels!=total_labels:
        total_labels = df_pred.pred_labels.apply(len).sum()
        df_pred.pred_labels = df_pred.pred_labels.apply(merge)
        new_total_labels = df_pred.pred_labels.apply(len).sum()

    # removing very short predicted spans (< 5 characters)
    df_pred.pred_labels = df_pred.pred_labels.apply(delet_short)

    ### computing scores
    # scores by span
    tpfp = df_pred.pred_labels.apply(len).sum()
    tpfn = df_pred.labels.apply(len).sum()
    tp = count_tp(df_pred.pred_labels, df_pred.labels)

    prec_span = tp/tpfp
    recall_span = tp/tpfn
    f1_span = harmonic_mean([prec_span, recall_span])

    dict_scores['prec_span'] = prec_span
    dict_scores['recall_span'] = recall_span
    dict_scores['f1_span'] = f1_span

    # scores by character

    def label_per_char(row):
        char_labels = [0]*len(row['sents'])
        labels = row['labels']
        for label in labels:
            char_labels[label[0]:label[1]] = [1]*(label[1]-label[0])
        return char_labels

    def pred_label_per_char(row):
        char_labels = [0]*len(row['sents'])
        labels = row['pred_labels']
        for label in labels:
            char_labels[label[0]:label[1]] = [1]*(label[1]-label[0])
        return char_labels

    df_pred['char_labels'] = df_pred.apply(label_per_char, axis=1)
    df_pred['char_pred_labels'] = df_pred.apply(pred_label_per_char, axis=1)

    scores_char = precision_recall_fscore_support(df_pred['char_labels'].sum(), df_pred['char_pred_labels'].sum())

    dict_scores['prec_char'] = scores_char[0][1]
    dict_scores['recall_char'] = scores_char[1][1]
    dict_scores['f1_char'] = scores_char[2][1]

    return dict_scores

## Run randomly 20 trainings of a LSTM model

In [120]:
list_scores = []

for _ in tqdm(range(20)):
    random_state = random.randint(0,10000)
    scores = run_all(random_state)
    list_scores.append(scores)

Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag : ▁explique ▁un ▁conseiller ▁du ▁chef ▁de ▁l ▁ ' ▁Etat ▁ . not found
Tag : ▁a - t - il ▁récemment ▁ ful miné ▁devant ▁un ▁proche . not found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag : ▁x plique ▁un ▁de ▁ses ▁amis . not found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly 

Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag : ▁Même ▁au ▁sein ▁de ▁son ▁camp , ▁certains ▁redoute nt ▁un ▁vote ▁sanction not found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag:  correctly found
Tag : ▁ er tain s ▁conseillers ▁ne ▁c ach aient ▁pas ▁leur ▁ amertume . not found
Tag : ▁e connais s ait ▁l ▁ ' ▁un ▁d ▁ ' ▁entre ▁eux not found
Tag : ▁ er cy ▁assure ▁que ▁ , not fou

Using gold standard
Epoch 1
-------------------------------
loss: 0.668885  [    0/ 6274]
loss: 0.130174  [ 5280/ 6274]


/pbs/home/r/rshen/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/pbs/home/r/rshen/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/pbs/home/r/rshen/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.97      1.00      0.99     75093
           1       0.00      0.00      0.00      2190

    accuracy                           0.97     77283
   macro avg       0.49      0.50      0.49     77283
weighted avg       0.94      0.97      0.96     77283



...

/pbs/home/r/rshen/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/pbs/home/r/rshen/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/pbs/home/r/rshen/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.97      1.00      0.99     75093
           1       0.00      0.00      0.00      2190

    accuracy                           0.97     77283
   macro avg       0.49      0.50      0.49     77283
weighted avg       0.94      0.97      0.96     77283



/pbs/home/r/rshen/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 4
-------------------------------
loss: 0.080510  [    0/ 6274]
loss: 0.067086  [ 5280/ 6274]
              precision    recall  f1-score   support

           0       0.97      1.00      0.99     75093
           1       0.75      0.11      0.19      2190

    accuracy                           0.97     77283
   macro avg       0.86      0.55      0.59     77283
weighted avg       0.97      0.97      0.96     77283

Epoch 5
-------------------------------
loss: 0.062922  [    0/ 6274]
loss: 0.060726  [ 5280/ 6274]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     75093
           1       0.72      0.25      0.37      2190

    accuracy                           0.98     77283
   macro avg       0.85      0.62      0.68     77283
weighted avg       0.97      0.98      0.97     77283

Epoch 6
-------------------------------
loss: 0.063555  [    0/ 6274]
loss: 0.050924  [ 5280/ 6274]
              precision    recall  f1-score   

/pbs/home/r/rshen/.local/lib/python3.7/site-packages/pandas/core/generic.py:5491: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/pbs/throng/humanum/anaconda/2020.02/lib/python3.7/site-packages/ipykernel_launcher.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/pbs/throng/humanum/anaconda/2020.02/lib/python3.7/site-packages/ipykernel_launcher.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

In [121]:
df_scores = pd.DataFrame(list_scores)

In [122]:
df_scores.to_csv('./train/lstm_scores.csv', index=False)

## Computing performance scores

In [123]:
df_scores = pd.read_csv('./train/lstm_scores.csv')

In [124]:
recap = pd.concat([pd.DataFrame(df_scores.mean()).transpose(), df_scores.agg(lambda g: sms.DescrStatsW(g).tconfint_mean())])
recap = recap.drop('random_state', axis=1)
recap.index = ['mean', '2.5% CI', '97.5% CI']

recap

,prec_span,recall_span,f1_span,prec_char,recall_char,f1_char
mean,0.457814,0.378246,0.413211,0.702791,0.348408,0.464207
2.5% CI,0.439085,0.350166,0.389574,0.687232,0.330070,0.447960
97.5% CI,0.476543,0.406325,0.436848,0.718349,0.366747,0.480455
